In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import nltk
from nltk import word_tokenize

In [2]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
    print(tree) # the grammar lets us look at the sentence in 2 ways

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [3]:
# a simple context free grammar.
# we can write our own grammars with CFG
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [4]:
# a grammar is said to be recursive if a category occuring on the left side also appears on the right side
# like Nom -> Adj Nom | N below
grammar2 = nltk.CFG.fromstring("""
  S  -> NP VP
  NP -> Det Nom | PropN
  Nom -> Adj Nom | N     
  VP -> V Adj | V NP | V S | V NP PP
  PP -> P NP
  PropN -> 'Buster' | 'Chatterer' | 'Joe'
  Det -> 'the' | 'a'
  N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
  Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
  V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
  P -> 'on'
  """)

In [5]:
# Parsing with context free grammar
# we'll see 2 simple parsing algorithms - recursive descent parsing and shift reduce parsing
# Recursive descent parsing 
# the top level goal is to find S. The S -> NP VP permits the parser to replace this goal with 2 subgoals
# find and NP, then find a VP. Each of these can be replaces by sub-sub goals.
rd_parser = nltk.RecursiveDescentParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [6]:
# Recursive descent parsing has three key shortcomings. 
# First, left-recursive productions like NP -> NP PP send it into an infinite loop. 
# Second, the parser wastes a lot of time considering words and structures that do not correspond to the input sentence. 
# Third, the backtracking process may discard parsed constituents that will need to be rebuilt again later.
# top down parsers use the grammar to predict what the input will be before inspecting the input

# a better approach is bottom up parsing like a shift-reduce parser
# It tries to find sequences of words and phrases that correspond to the right hand side of a grammar production, 
# and replace them with the left-hand side, until the whole sentence is reduced to an S.
sr_parser = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
    print(tree)
    
# A shift-reduce parser can reach a dead end and fail to find any parse. when this happens, no inputs remain and the
# stack contains items which cannot be reduced to an S

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [7]:
# the left corner parser
# this is a hybrid between the top down and bottom up approaches we have seen
# it is a top down parser with bottom up filtering

In [8]:
# Well formed substring tables
# we will apply the algorith design technique of dynamic programming to store the results of sub-problems and 
# reuse them when appropriate to achieve significant efficiency gains. In the case of syntactic parsing, we can store
# partial solutions to the parsing task and then look them up as necessary in order to efficiently arrive at 
# a complete solution. This approach is also known as chart parsing
# we need to build PP in my pajamas only once and just look it up when we need to use it as a subconstituent of either
# the object NP or the higher VP. This table is known as a well formed substring table (in my pajamas is a substring)
# for every word in text, we can look up in our grammar which category it belongs to 
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
groucho_grammar.productions(rhs=text[1])

[V -> 'shot']

In [16]:
# For our WFST, we create an (n-1)*(n-1) matrix as a list of lists in python and initialize it with the lexical 
# categories of each token and even define a funtion display() to pretty-print the WFST for us
def init_wsft(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst

def display(wfst, tokens):
    print('\nWFST ' + ' '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

In [17]:
tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wsft(tokens, groucho_grammar)

In [18]:
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    


In [19]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


In [21]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
